# Analysis of the influence of geomechanical parameters and geometry on slope slope stability in granitic residual soils

Notebook for model creation and calculation automatization

## 1.- Import dependencies

In [1]:
import math
from numpy import arange, array, deg2rad, tan
from numpy import random
from numpy.lib.function_base import append
import xml.etree.ElementTree as ET

## 2.- Definition of variables and Montecarlo simulation

In [6]:
# number of simulations
n = 10
        
# Montecarlo simulation - (min, máx, number of simulations)
# Generates the set of variables using uniform probaility.

# Slope geometry
# Slope height
h = random.uniform(2, 30, n)
h = h.tolist()

# Slope angle
b = random.uniform(26, 80, n) 
b = b.tolist()

# Strenght parameters
# Cohesion
c = random.uniform(0, 5, n)  
c = c.tolist()

# Friction angle
phi = random.uniform(29.5, 38, n)
phi = phi.tolist()

# Unit weight
gamma = random.uniform(17, 21, n)
gamma = gamma.tolist()


file_list = []  #list to save file numbers (example: model_0)
file_results = [] #list to save file results numbers (example: model_results_0)

## 3.- Template configuration

In [ ]:
id_material  = 101   #id material - in this case : weathered granite soils
id_node_h = 3   #id node that changes the height
id_node_beta = 4 #id node that changes the angle of slope

## 4.- Model creation

Changes the values of the parameters defined using Montecarlo's simulation and generate the *.g2x* files

In [7]:
# Read OptumG2 template file file
tree = ET.parse('template_SRF_AS.g2x')  #name of .g2x file
# Obtain the root element
root = tree.getroot()

In [8]:
for i in range(n):
    
    b_radians = deg2rad(b[i])  # slope angle degree to radians
    coord_x = (h[i]/tan(b_radians)) #new x coord
    
    list_coords_3 = [80, h[i], 0] #new coords node 3 - [x coord template, new y coord, z coord]
    list_coords_4 = [coord_x, h[i], 0] #new coords node 4 - [new x coord, new y coord, z coord]

    list_coords_str3 = [str(coordenada) for coordenada in list_coords_3] #coords to string
    list_coords_str4 = [str(coordenada) for coordenada in list_coords_4] #coords to string

    coordenadas_str3 = ','.join(list_coords_str3) 
    coordenadas_str4 = ','.join(list_coords_str4)
    
    # Change template coords
    
    nodo3 = root.findall("./Stages/Stage//Data/Nodes/Node[@ID='{}']".format(id_node_h))[0]
    nodo3.text = coordenadas_str3

    nodo4 = root.findall("./Stages/Stage//Data/Nodes/Node[@ID='{}']".format(id_node_beta))[0]
    nodo4.text = coordenadas_str4
    
    # Change template parameters
    
    cohesion = c[i]
    cohesion_new  = root.findall(".//*Material[@ID='{}']/Parameters/c".format(id_material))[0]
    cohesion_new.text = str(cohesion)
    
    friction_angle = str(phi[i])
    friction_angle_new = root.findall(".//*Material[@ID='{}']/Parameters/Phi".format(id_material))[0]
    friction_angle_new.text = friction_angle
       
    specific_gravity = str(gamma[i])
    specific_gravity_new = root.findall(".//*Material[@ID='{}']/Parameters/GammaDry".format(id_material))[0]
    specific_gravity_new.text = specific_gravity
    
    #Create new files and files results
    
    model = 'model_' + str(i) + '.g2x'
    result = 'model_' + str(i) + '.results.g2x'
    
    tree.write(model)
    
    #Save files and files results numbers
    
    file_list.append(model)
    file_results.append(result)
    

## 5.- Run vreated OptumG2 files

In [ ]:
prog = 'OptumG2Cmd '

for file in file_list:
    command = prog + file
    display(command)
    !$command

## 6.- Save results

In [6]:
SRF_results = []

for results in file_results:
    tree = ET.parse(results)
    root = tree.getroot()
    
    for Result in root.findall(".//Result[2]"):
        srf = Result.text
        srf_value = eval(srf)     
        SRF_results.append(srf_value) 
        
#display(SRF_results)

In [7]:
#export results in .txt file
output_LB = "Output_LB.txt" #name of output1 file
output_UB = "Output_UB.txt" #name of outpu2 file
entrada = "Input.txt"       #name of input file

#Export input data 
with open(entrada, 'a+') as f:
    f.write('\n')
with open(output_LB, 'a+') as f:
    f.write('\n')
with open(output_UB, 'a+') as f:
    f.write('\n')

for i in range(n):
    with open(entrada, 'a+') as f:
        f.write(str(h[i]) + ' ' + 
                str(b[i]) + ' ' + 
                str(c[i]) + ' ' + 
                str(phi[i]) + ' ' +  
                str(gamma_dry[i]) + '\n')


#Export output data
for i in range(len(SRF_results)):
    if i%2 == 0:
        with open(output_LB, 'a+') as f:
            f.write(str(SRF_results[i]) + '\n')
    elif i%2 != 0:
        with open(output_UB, 'a+') as f:
            f.write(str(SRF_results[i]) + '\n')        
    else:
        print('error')